# 타율 예측

**직전 50경기 타율들과 평균타율, 승률, 방어율로 이후 20경기 예측하기

    train set : 14-63 / 24-83 / 54-103 번째 경기
    test set  : 74-123 번째 경기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1. 데이터 가져오기

In [2]:
t_bat16=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2016.csv",encoding='utf-8',engine='python')
t_bat17=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2017.csv",encoding='utf-8',engine='python')
t_bat18=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2018.csv",encoding='utf-8',engine='python')
t_bat19=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2019.csv",encoding='utf-8',engine='python')

In [3]:
t_pit16=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2016.csv",encoding='utf-8',engine='python')
t_pit17=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2017.csv",encoding='utf-8',engine='python')
t_pit18=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2018.csv",encoding='utf-8',engine='python')
t_pit19=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2019.csv",encoding='utf-8',engine='python')

In [4]:
#모든 열 다 출력하기
pd.options.display.max_columns=None

## 2. train test 셋 만들기

In [5]:
team=['HH', 'HT', 'KT', 'LG', 'LT', 'NC', 'OB', 'SK', 'SS', 'WO']

#### 승률 변수 만들기

In [6]:
pd.set_option('mode.chained_assignment',None) #chain copy warning 제거하기

def make_win_lose(data):
    data['W']=0; data['L']=0
    data['W'][data['WLS']=='W']=1
    data['L'][data['WLS']=='L']=1
    return data

In [7]:
#팀투수 데이터에 승,패 여부 추가
data_list=[t_pit16,t_pit17,t_pit18,t_pit19] 
for data in data_list:
    data=make_win_lose(data)

In [8]:
#팀타자 데이터에 승,패 변수 추가
t_bat16=pd.merge(t_bat16,t_pit16[['G_ID','T_ID','W','L','ER','INN2']],on=['G_ID','T_ID'])
t_bat17=pd.merge(t_bat17,t_pit17[['G_ID','T_ID','W','L','ER','INN2']],on=['G_ID','T_ID'])
t_bat18=pd.merge(t_bat18,t_pit18[['G_ID','T_ID','W','L','ER','INN2']],on=['G_ID','T_ID'])
t_bat19=pd.merge(t_bat19,t_pit19[['G_ID','T_ID','W','L','ER','INN2']],on=['G_ID','T_ID'])

#### 타율 변수 만드는 함수

In [9]:
def make_data(train_data,answer_data):
    train_data['HIT_RATE']=train_data['HIT']/train_data['AB']
    tr=train_data[['T_ID','HIT_RATE']]                          #경기당 팀과 타율만 남김
    num=list(range(1,int((len(train_data)/10)+1)))*10           #팀별 경기순서 추가
    tr=tr.assign(order=num)
    tr=tr.pivot(index='T_ID',columns='order',values='HIT_RATE') #팀당 경기별 타율 데이터셋 만들기
    
    data=pd.merge(tr,answer_data,on='T_ID')                     #팀별 앞124경기와 후20경기 결과 합치기
    return data

#### train, test set 만드는 함수

In [10]:
t_bat=[t_bat16,t_bat17,t_bat18,t_bat19]
def make_answer_train(year):
    a=t_bat[year-2016][['T_ID','HIT','AB','W','L','ER','INN2']]  
    data=pd.DataFrame()  #빈 데이터프레임 만들기
    pa=pd.DataFrame()    #빈 데이터프레임 만들기

    train_data=[pd.DataFrame() for var in range(1,5)]   #50경기씩 묶일 데이터프레임
    answer=[pd.DataFrame() for var in range(1,5)]       #최종 후반 20경기 묶일 데이터프레임
        
    for i in range(4):
        tra=train_data[i]
        ans=answer[i]
        for t in team:
            tra=pd.concat([tra,a[a['T_ID']==t][14+i*20:14+50+i*20]])     #각 팀별 앞 14경기 제외하고, 50경기씩 묶음(train set)
            ans=pd.concat([ans,a[a['T_ID']==t][14+50+i*20:14+70+i*20]])  #train set의 경기의 다음 20경기 묶음(test set)
    
        #팀별 AB,HIT합 구하기
        ans=ans.groupby(['T_ID']).sum()
        ans['HIT_RATE']=ans['HIT']/ans['AB']              #타율 계산
        ans=ans.drop(ans.columns[list(range(6))],axis='columns')
        
        #변수 생성
        tr=tra.groupby(['T_ID']).sum() 
        tr['WINNING_RATE'] = tr['W']/(tr['W']+tr['L'])    #승률 변수생성
        tr['ERA'] = tr['ER']*27/tr['INN2']                #방어율 변수생성
        
        #변수 삭제
        tr=tr.drop(['HIT','W','L','ER','INN2'],axis='columns')
        
        if i <3:
            data=pd.concat([data,make_data(tra,ans)])  
            pa=pd.concat([pa,tr])
        else:
            test_data=make_data(tra,ans)
            test_data['WINNING_RATE']=tr['WINNING_RATE']   #test set에 승률 추가
            test_data['ERA']=tr['ERA']                     #test set에 ERA 추가
            
    #변수 추가
    data['MEAN']=data.iloc[:,:50].mean(axis=1)             #train set에 평균 추가
    test_data['MEAN']=test_data.iloc[:,:50].mean(axis=1)   #test set 에 평균 추가
    data['WINNING_RATE']=pa['WINNING_RATE']                #train set에 승률 추가
    data['ERA']=pa['ERA']                                  #train set에 ERA 추가
        
    return data,test_data

In [11]:
#train, test set 만들기
train16,test16=make_answer_train(2016)
train17,test17=make_answer_train(2017)
train18,test18=make_answer_train(2018)
train19,test19=make_answer_train(2019)

In [12]:
train16

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,HIT_RATE,MEAN,WINNING_RATE,ERA
T_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
HH,0.277778,0.393939,0.212121,0.181818,0.129032,0.222222,0.236842,0.394737,0.225806,0.285714,0.178571,0.218750,0.333333,0.270270,0.324324,0.133333,0.361111,0.181818,0.187500,0.166667,0.428571,0.250000,0.187500,0.270270,0.352941,0.319149,0.275000,0.156250,0.378378,0.342105,0.410256,0.333333,0.312500,0.400000,0.285714,0.303030,0.200000,0.324324,0.250000,0.275862,0.333333,0.161290,0.250000,0.242424,0.323529,0.205882,0.324324,0.366667,0.297297,0.257143,0.307692,0.274655,0.469388,5.934734
HT,0.205882,0.277778,0.437500,0.357143,0.225806,0.175000,0.233333,0.297297,0.218750,0.241379,0.233333,0.473684,0.371429,0.193548,0.297297,0.368421,0.421053,0.222222,0.333333,0.281250,0.305556,0.272727,0.314286,0.321429,0.242424,0.312500,0.250000,0.272727,0.375000,0.270270,0.315789,0.297297,0.204545,0.272727,0.206897,0.156250,0.312500,0.225806,0.129032,0.363636,0.258065,0.129032,0.342857,0.324324,0.305556,0.297297,0.352941,0.121212,0.333333,0.371429,0.303644,0.282458,0.428571,5.310475
KT,0.303030,0.228571,0.400000,0.390244,0.315789,0.200000,0.250000,0.103448,0.258065,0.250000,0.264706,0.290323,0.257143,0.218750,0.428571,0.476190,0.393939,0.264706,0.258065,0.277778,0.264706,0.166667,0.371429,0.161290,0.096774,0.242424,0.295455,0.468085,0.282051,0.375000,0.305556,0.314286,0.212121,0.272727,0.285714,0.142857,0.235294,0.236842,0.257143,0.297297,0.133333,0.343750,0.241379,0.340000,0.235294,0.068966,0.214286,0.290323,0.161290,0.222222,0.279661,0.267278,0.395833,5.316062
LG,0.307692,0.161290,0.222222,0.294118,0.218750,0.305556,0.285714,0.178571,0.250000,0.129032,0.277778,0.172414,0.193548,0.250000,0.272727,0.488889,0.303030,0.275862,0.421053,0.264706,0.281250,0.210526,0.310345,0.333333,0.241379,0.233333,0.235294,0.277778,0.212121,0.465116,0.212766,0.285714,0.354839,0.292683,0.441860,0.307692,0.352941,0.382353,0.371429,0.200000,0.342105,0.342857,0.350000,0.187500,0.205882,0.363636,0.333333,0.325000,0.181818,0.181818,0.292750,0.281793,0.428571,5.314801
LT,0.172414,0.447368,0.272727,0.393939,0.350000,0.222222,0.166667,0.277778,0.333333,0.233333,0.156250,0.156250,0.166667,0.323529,0.103448,0.324324,0.264706,0.477273,0.264706,0.300000,0.378378,0.243243,0.289474,0.305556,0.205882,0.264706,0.297297,0.350000,0.394737,0.275862,0.305556,0.333333,0.290323,0.218750,0.333333,0.285714,0.222222,0.250000,0.297297,0.307692,0.218750,0.241379,0.264706,0.323529,0.214286,0.277778,0.277778,0.378378,0.205882,0.242424,0.310576,0.278024,0.420000,6.145675
NC,0.333333,0.368421,0.222222,0.193548,0.166667,0.343750,0.235294,0.272727,0.289474,0.277778,0.361111,0.372093,0.405405,0.250000,0.400000,0.257143,0.333333,0.257143,0.277778,0.162791,0.250000,0.388889,0.297297,0.235294,0.393939,0.378378,0.285714,0.270270,0.439024,0.410256,0.368421,0.235294,0.181818,0.250000,0.133333,0.386364,0.241379,0.266667,0.375000,0.351351,0.292683,0.400000,0.368421,0.218750,0.400000,0.390244,0.468085,0.133333,0.175000,0.218750,0.265861,0.299679,0.708333,4.152318
OB,0.400000,0.264706,0.382353,0.241379,0.178571,0.241379,0.257143,0.300000,0.166667,0.312500,0.257143,0.437500,0.422222,0.161290,0.200000,0.391304,0.351351,0.342857,0.285714,0.384615,0.428571,0.297297,0.300000,0.439024,0.303030,0.395349,0.333333,0.257143,0.290323,0.406250,0.312500,0.290323,0.290323,0.257143,0.222222,0.225806,0.166667,0.300000,0.290323,0.272727,0.309524,0.242424,0.297297,0.200000,0.250000,0.295455,0.305556,0.333333,0.375000,0.205882,0.284672,0.297430,0.700000,4.334584
SK,0.205882,0.361111,0.222222,0.230769,0.351351,0.290323,0.342105,0.187500,0.258065,0.303030,0.333333,0.166667,0.242424,0.322581,0.437500,0.312500,0.225806,0.348837,0.342105,0.281250,0.303030,0.100000,0.147059,0.281250,0.225806,0.250000,0.235294,0.324324,0.285714,0.038462,0.418605,0.312500,0.250000,0.289474,0.312500,0.384615,0.133333,0.137931,

## 3. 모델학습(Random Forest)

In [12]:
# 모델 모듈
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [13]:
X_cols=list(range(1,51))+['MEAN','WINNING_RATE','ERA']
Y_col=['HIT_RATE']

In [14]:
train_list=[train16,train17,train18,train19]
test_list=[test16,test17,test18,test19]

In [15]:
#4개년 랜포 돌려보기
result=[]
for i in range(4):
    train=train_list[i];  test=test_list[i]
    X_train = train[X_cols]; Y_train = train[Y_col]
    X_test = test[X_cols]; Y_test = test[Y_col]

    model = RandomForestRegressor(random_state=1234)
    Y_train=Y_train.values.ravel()          #rf에서의 오류 없애기 위함
    model.fit(X_train, Y_train) 
    Y_predict = model.predict(X_test)

    r2=r2_score(Y_test, Y_predict) 

    result.append(Y_test.assign(predict=Y_predict))
    result[i]['diff']=result[i]['HIT_RATE']-result[i]['predict']
    print(i+2016,">\tr2 :", r2,"\t차이합 :",abs(result[i]['diff']).sum())

2016 >	r2 : -0.34724847366520306 	차이합 : 0.09793047322286585
2017 >	r2 : 0.10502049910785749 	차이합 : 0.11828556352453623
2018 >	r2 : 0.2921787736500143 	차이합 : 0.12427250371235726
2019 >	r2 : -1.3811789824832128 	차이합 : 0.2252243315141516


### 예측값과 실제값 차이

In [16]:
print('2016 >',abs(result[0]['diff']).sum())
result[0]

2016 > 0.09793047322286585


,HIT_RATE,predict,diff
T_ID,,,
HH,0.291317,0.301063,-0.009747
HT,0.263400,0.298136,-0.034736
KT,0.288630,0.294050,-0.005420
LG,0.289706,0.296720,-0.007014
LT,0.314448,0.286100,0.028348
NC,0.291429,0.294071,-0.002642
OB,0.304654,0.301840,0.002814
SK,0.300562,0.298544,0.002018
SS,0.285311,0.287198,-0.001887


In [17]:
print('2017 >',abs(result[1]['diff']).sum())
result[1]

2017 > 0.11828556352453623


,HIT_RATE,predict,diff
T_ID,,,
HH,0.275568,0.294748,-0.019180
HT,0.298295,0.295506,0.002789
KT,0.297486,0.288253,0.009233
LG,0.271533,0.282038,-0.010505
LT,0.296296,0.291901,0.004395
NC,0.331579,0.291498,0.040081
OB,0.295105,0.288795,0.006310
SK,0.297450,0.295501,0.001949
SS,0.290323,0.286012,0.004311


In [18]:
print('2018 >',abs(result[2]['diff']).sum())
result[2]

2018 > 0.12427250371235726


,HIT_RATE,predict,diff
T_ID,,,
HH,0.276878,0.283936,-0.007058
HT,0.279830,0.287772,-0.007942
KT,0.287879,0.293752,-0.005874
LG,0.267908,0.286285,-0.018377
LT,0.300699,0.288046,0.012653
NC,0.271408,0.283710,-0.012302
OB,0.315646,0.300869,0.014778
SK,0.281471,0.291455,-0.009984
SS,0.306207,0.296996,0.009211


In [19]:
print('2019 >',abs(result[3]['diff']).sum())
result[3]

2019 > 0.2252243315141516


,HIT_RATE,predict,diff
T_ID,,,
HH,0.266764,0.264463,0.002301
HT,0.253373,0.268567,-0.015194
KT,0.272321,0.267121,0.005200
LG,0.257485,0.273886,-0.016401
LT,0.220877,0.271000,-0.050123
NC,0.275109,0.267633,0.007476
OB,0.281690,0.269475,0.012215
SK,0.239695,0.290985,-0.051290
SS,0.232061,0.277658,-0.045596
